In [6]:
import os
import sys
import pandas as pd
from robobrowser import RoboBrowser

In [61]:
# 認証情報の取得
email = pd.read_csv('amazon_login.csv', header=None, names=['name','value'])
AMAZON_EMAIL = email.loc[0, 'value']
AMAZON_PASSWORD = email.loc[1, 'value']

In [62]:
# RoboBrowserオブジェクトを作成する。
browser = RoboBrowser(
    parser = 'html.parser',  # Beautiful Soupで使用するパーサーを指定する。
    # Cookieが使用できないと表示されたログインできない問題を回避するため、
    # 通常のブラウザーのUser-Agent(ここではFirefoxのもの)を使う。
    user_agent='Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:58.0) Gecko/20100101 Firefox/58.0')

In [63]:
def main():
    # 注文履歴のページを開く。
    print('Navigating...', file=sys.stderr)
    browser.open('https://www.amazon.co.jp/gp/css/order-history')
    
    # サインページにリダイレクトされていることを確認する。
    assert 'Amazonサインイン' in browser.parsed.title.string
    
    # name='singIn'というサインフォームを埋める。
    # フォームのname属性の値はブラウザーの開発者ツールで確認できる。
    form = browser.get_form()
    form['email'] = AMAZON_EMAIL  # name='email'というボックスを埋める。
    form['password'] = AMAZON_PASSWORD  # name='password'という入力ボックスを埋める。
    
    # フォームを送信する。正常にログインするにはRefereヘッダーとAccept-Languageヘッダーが必要。
    print('Sign in...', file=sys.stderr)
    browser.submit_form(form, headers={
        'Referer':browser.url,
        'Accept-Language':'ja,en-US;q=0.7,en;q=0.3'
    })
    
    # ログインに失敗する場合は、次の行のコメントを外してHTMLのソースを確認すると良い。
    # print(browser.parsed.prettify())
    
    # ページャーをたどる。
    while True:
        assert '注文履歴' in browser.parsed.title.string  # 注文履歴画面が表示されていることを確認する。
        
        print_order_history()  # 注文履歴を表示する。
        
        link_to_next = browser.get_link('次へ')  # 「次へ」というテキストを持つリンクを取得する。
        if not link_to_next:
            break  # 「次へ」のリンクがない場合はループを抜けて終了する。
        
        print('Following link to next page...', file=sys.stderr)
        browser.follow_link(link_to_next)  # 「次へ」というリンクをたどる。

In [64]:
def print_order_history():
    """
    現在のページの全ての注文履歴を表示する。
    """
    # ページ内の全ての注文履歴について反復する。ブラウザーの開発者ツールでclass属性の値を確認できる。
    for line_item in browser.select('.order-info'):
        order = {}  # 注文の情報を格納するためのdict。
        # 注文の情報の全ての列について反復する。
        for column in line_item.select('.a-column'):
            label_element = column.select_one('.label')
            value_element = column.select_one('.value')
            # ラベルと値がない列は無視する。
            if label_element and value_element:
                label = label_element.get_text().strip()
                value = value_element.get_text().strip()
                order[label] = value  # 注文の情報を格納する。
                
        print(order['注文日'], order['合計'])  # 注文の情報を表示する。
    

In [65]:
if __name__=='__main__':
    main()

Navigating...
Sign in...


2018年4月22日 ￥ 1,567
2018年4月15日 ￥ 842
2018年4月14日 ￥ 4,104
2018年4月13日 ￥ 1,944
2018年4月13日 ￥ 3,024
2018年4月13日 ￥ 3,024
2018年4月12日 ￥ 0
2018年4月11日 ￥ 3,269
2018年4月9日 ￥ 2,862
2018年4月8日 ￥ 12,096


Following link to next page...


2018年4月8日 ￥ 6,242
2018年4月6日 ￥ 3,748
2018年4月3日 ￥ 3,024
2018年4月3日 ￥ 565
2018年3月28日 ￥ 7,344
2018年3月28日 ￥ 8,316
2018年3月26日 ￥ 1,296
2018年3月26日 ￥ 1,567
2018年3月26日 ￥ 5,141
2018年3月26日 ￥ 4,860


Following link to next page...


...

2018年3月9日 ￥ 1,296
2018年3月6日 ￥ 3,748
2018年2月23日 ￥ 1,567
2018年2月22日 ￥ 0
2018年2月6日 ￥ 3,748
2018年1月26日 ￥ 1,377
2018年1月20日 ￥ 2,916
2018年1月18日 ￥ 3,910
2018年1月17日 ￥ 3,888
2018年1月6日 ￥ 3,748


Following link to next page...


2018年1月3日 ￥ 6,598
2017年12月29日 ￥ 1,511
2017年12月27日 ￥ 3,456
2017年12月27日 ￥ 1,182
2017年12月26日 ￥ 1,377
2017年12月25日 ￥ 4,290
2017年12月12日 ￥ 994
2017年11月26日 ￥ 1,187
2017年11月25日 ￥ 1,472
2017年11月19日 ￥ 1,393


Following link to next page...


2017年11月18日 ￥ 3,748
2017年11月12日 ￥ 1,187
2017年11月7日 ￥ 1,836
2017年11月3日 ￥ 623
2017年11月2日 ￥ 2,766
2017年10月28日 ￥ 1,420
